<a href="https://colab.research.google.com/github/sukmoonlee/JupyterNotebook/blob/main/2021/DellEMC_Firmware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DellEMC 서버 펌웨어 추출하기
 - http://www.poweredgec.com/ 에서 최신 펌웨어 정보 추출

 

### 필요 라이브러리 및 데이터 다운로드

In [1]:
import pandas as pd
#import urllib.request as req
from bs4 import BeautifulSoup
import datetime

In [2]:
## Dell PowerEdge 14G : http://www.poweredgec.com/latest_poweredge-14g.html
## Dell PowerEdge 13G : http://www.poweredgec.com/latest_poweredge-13g.html"

%%capture
!wget "http://www.poweredgec.com/latest_poweredge-14g.html" -O latest_poweredge.html
!sed 's/<\/td><th>/<\/th><th>/g' latest_poweredge.html > latest.html

soup = BeautifulSoup(open("latest.html"), "html.parser")

### 펌웨어 정보 추출하기 (R640/R740 리눅스용)

In [3]:
## 타이틀(모델 펌웨어 유형) 읽어오기
models = []
rows = soup.select("h2.platformhead > a")
for row in rows:
  title = row.string
  models.append(title)

In [4]:
dell_df = pd.DataFrame()
cnt = 0
idx = 0
tables = soup.select("table.fwtable")
for table in tables:
  
  # 추출 모델 정하기
  flag = 0
  if models[cnt][0:5] == "R640 ":
    flag = 1
  elif models[cnt][0:5] == "R740 ":
    flag = 1
  elif models[cnt][0:5] == "R630 ":
    flag = 1
  elif models[cnt][0:5] == "R730 ":
    flag=1
  if flag == 0:
    cnt = cnt + 1
    continue
 
  headers = []
  table_datas = table.select("tr")  
  for table_data in table_datas:
    # 헤더 추출하기
    ths = table_data.select("th")
    if len(ths) != 0 :
      for th in ths:
        headers.append(th.string)   
      continue

    d = pd.DataFrame()
    flag = 0
    idx = idx + 1
    d.loc[idx, "model" ] = models[cnt]
    field_cnt=0
    tds = table_data.select("td")    
    for td in tds:
      d.loc[idx, headers[field_cnt] ] = td.string

      if headers[field_cnt] == "Filename":        
        #href = td.find_all("a", href=True)
        href = td.select("a")
        d.loc[idx, "UrlFilename" ] = href[0]['href']

        if td.string[-5:] == ".sign":
          flag=1

      if headers[field_cnt] == "DUP?" and td.string != "linux dup":  
          flag = 1
      if flag == 1:
        break
      field_cnt = field_cnt + 1
    
    if flag==1:
      idx = idx - 1
    else:
      dell_df = dell_df.append(d)

  cnt = cnt + 1
  #break

dell_df

,model,Rel Date,Filename,UrlFilename,Importance,Ver,Rev,Latest?,DUP?,Description
1,R640 BIOS,2020-12-08,BIOS_NVGR9_LN_2.10.0.BIN,https://downloads.dell.com/FOLDER06792461M/3/B...,Recommended,2.10.0,2.10.0,NaN,linux dup,Dell EMC Server PowerEdge BIOS R740/R740xd/R64...
2,R640 CPLD,2020-01-21,CPLD_Firmware_9N4DH_LN_1.0.6_A00.BIN,https://downloads.dell.com/FOLDER05667811M/1/C...,Recommended,1.0.6,A00,NaN,linux dup,PowerEdge CPLD R640 Version 1.0.6
3,R640 Diags,2020-12-22,Diagnostics_Application_0G1JH_LN64_6.0_A00_01.BIN,https://downloads.dell.com/FOLDER06878161M/2/D...,Recommended,6.0,A00,NaN,linux dup,OS COLLECTOR v6.0
4,R640 Diags,2020-12-21,Diagnostics_Application_Y9DR5_LN_4301A50_4301....,https://downloads.dell.com/FOLDER06557528M/1/D...,Optional,4301A50,4301.51,NaN,linux dup,Dell 64 Bit uEFI Diagnostics version 4301
5,R640 Diags,2020-10-06,Diagnostics_Application_D16GN_LN_4301A49_4301....,https://downloads.dell.com/FOLDER06580215M/1/D...,Optional,4301A49,4301.50,NaN,linux dup,Dell 64 Bit uEFI Diagnostics version 4301
...,...,...,...,...,...,...,...,...,...,...
274,R740 Solid State Storage,2020-01-06,Express-Flash-PCIe-SSD_Firmware_90R8R_LN64_VDV...,https://downloads.dell.com/FOLDER05393672M/1/E...,Recommended,VDV1DP21,A00,NaN,linux dup,Dell Express Flash NVMe P4510/P4610 PCIe SSD F...
275,R740 Solid State Storage,2020-02-18,Express-Flash-PCIe-SSD_Firmware_KNW4M_LN64_QDV...,https://downloads.dell.com/FOLDER05152843M/2/E...,Urgent,QDV1DP15,A02,NaN,linux dup,Dell Express Flash NVMe P4500/P4600 PCIe SSD F...
276,R740 Storage,2020-08-07,Storage_Firmware_2NPVN_LN_1.13_A04.BIN,https://downloads.dell.com/FOLDER06190369M/1/S...,Recommended,1.13,A04,NaN,linux dup,Dell 14G /15G Internal Dual SD Module
277,R740 iDRAC with Lifecycle controller,2020-12-09,iDRAC-with-Lifecycle-Controller_Firmware_62GW1...,https://downloads.dell.com/FOLDER06841001M/1/i...,Recommended,4.40.00.00,A00,NaN,linux dup,iDRAC 4.40.00.00


### 펌웨어 정보 저장하기

In [5]:
## 엑셀저장하기
dell_df.to_excel("DELL_Firmware.xls")

In [6]:
## bash 코드 저장하기
now = datetime.datetime.now()

print("################################################################################")
print("# Dell Firmware Latest Information auto-generation (%s)" % now.strftime('%Y-%m-%d'))
print("LATEST_DATE=\"%s\"" % now.strftime('%Y-%m-%d'))
print("")

last_model=""
for idx in dell_df.index:
  model = dell_df.loc[idx, "model"]
  model = model.replace(" ","_")
  model = model.replace("-","_")
  if model != last_model :
    if last_model != "" :
      print("'")
      print("")
    print("# shellcheck disable=SC2034")
    print("list_latest_%s='" % model)
    last_model = model
  
  dell_df.loc[idx, "Description"] = dell_df.loc[idx, "Description"].replace("'","")
  print("%s %s (%s %s) %s" %
        (dell_df.loc[idx, "Ver"] , 
         dell_df.loc[idx, "Rev"] ,
         dell_df.loc[idx, "Rel Date"],
         dell_df.loc[idx, "Importance"], 
         dell_df.loc[idx, "Description"] ) )
  
if last_model != "" :
  print("'")
  print("")

################################################################################
# Dell Firmware Latest Information auto-generation (2021-01-11)
LATEST_DATE="2021-01-11"

# shellcheck disable=SC2034
list_latest_R640_BIOS='
2.10.0 2.10.0 (2020-12-08 Recommended) Dell EMC Server PowerEdge BIOS R740/R740xd/R640/R940/7920R Version 2.10.0
'

# shellcheck disable=SC2034
list_latest_R640_CPLD='
1.0.6 A00 (2020-01-21 Recommended) PowerEdge CPLD R640 Version 1.0.6
'

# shellcheck disable=SC2034
list_latest_R640_Diags='
6.0 A00 (2020-12-22 Recommended) OS COLLECTOR  v6.0
4301A50 4301.51 (2020-12-21 Optional) Dell 64 Bit uEFI Diagnostics  version 4301
4301A49 4301.50 (2020-10-06 Optional) Dell 64 Bit uEFI Diagnostics  version 4301
4.0 A00 (2019-11-19 Recommended) OS COLLECTOR 14G
'

# shellcheck disable=SC2034
list_latest_R640_Drivers_for_OS_Deployment='
20.08.09 A00 (2020-09-29 Optional) Dell OS Driver Pack
19.04.05 A00 (2019-12-18 Optional) Dell OS Driver Pack
'

# shellcheck disable=SC2034
lis